# How to work with ERA5 Essential Climate Variables on Earth Data Hub
### Climatological analysis of temperature in Germany

***
This notebook will provide you guidance on how to access and use the `ecv-for-climate-change-1979-2023.zarr` datset on Earth Data Hub.

The first goal is to compute the 2 metre temperature anomaly in Winter 2022 over Central Europe.

The second goal is show the winter anomaly trend, for the available time period, by averaging the anomaly over the selected area.
***

## What you will learn:

* how to access and preview the dataset
* select and reduce the data
* plot the results

## Data access and preview
***

Xarray and Dask work together following a lazy principle. This means when you access and manipulate a Zarr store the data is in not immediately downloaded and loaded in memory. Instead, Dask constructs a task graph that represents the operations to be performed. A smart user will reduce the amount of data that needs to be downloaded before the computation takes place (e.g., when the `.compute()` or `.plot()` methods are called).

To preview the data, only the dataset metadata must be downloaded. Xarray does this automatically:

***

In [ ]:
import xarray as xr

# your `~/.netrc` file MUST contain your credentials for earthdatahub.com
#
# machine earthdatahub.com
#   login {your_username}
#   password {your_password}

ds = xr.open_dataset(
    "https://earthdatahub.com/stores/hedp/era5/ecv-for-climate-change-1979-2023.zarr",
    chunks={},
    engine="zarr",
    storage_options={"client_kwargs": {"trust_env": True}},
)
ds

## Working with data

Datasets on EDH are typically very large and remotely hosted. Typical use imply a selection of the data followed by one or more reduction steps to be performed in a local or distributed Dask environment. 

The structure of a workflow that uses EDH data looks like this:
1. data selection
2. (optional) data reduction
3. (optional) visualization

## 2 metre temperature: central Europe winter 2022 anomaly

### 1. Data selection

First, we select the 2 metre temperature anomaly from the ECV dataset. Then, we perform a geographical selection corresponding to the central Europe area. This greatly reduces the amount of data that will be downloaded from EDH.

In [ ]:
t2m_anomaly = ds.t2m
t2m_anomaly_central_europe = t2m_anomaly.sel(**{"latitude": slice(55, 45), "longitude": slice(2, 24)})
t2m_anomaly_central_europe

At this point, no data has been downloaded yet, nor loaded in memory. However, the selection is small enough to call `.compute()` on it. This will trigger the download of data from EDH and load it in memory. 

We can measure the time it takes:

In [ ]:
%%time

t2m_anomaly_central_europe = t2m_anomaly_central_europe.compute()

The selection was small, this didn't take long!

### 2. Data reduction

Now that the data is loaded in memory, we can easily compute the average 2 metre temperature anomaly by season:

In [ ]:
t2m_anomaly_central_europe_seasonal_mean = t2m_anomaly_central_europe.resample(time="QS-DEC").mean(dim="time")

And the winter 2022 anomaly:

In [ ]:
t2m_anomaly_central_europe_winter_2022_mean = t2m_anomaly_central_europe_seasonal_mean.sel(time=f"2022-{12}")

### 3. Visualization

We can plot the average winter 2022 anomaly on a map:

In [ ]:
from cartopy import crs, feature
import matplotlib.pyplot as plt

_, ax = plt.subplots(
    figsize=(6, 6),
    subplot_kw={"projection": crs.Miller()},
)
t2m_anomaly_central_europe_winter_2022_mean.plot(
    ax=ax, 
    cmap="YlOrRd", 
    transform=crs.PlateCarree(), 
    cbar_kwargs={"orientation": "horizontal", "pad": 0.05, "aspect": 40, "label": "Surface Temperature anomaly [C]"},
)
ax.coastlines()
ax.add_feature(feature.BORDERS)
ax.set_title("Surface Temperature anomaly - Winter 2022")
plt.show()

## 2 metre temperature: central Europe winter anomaly trend (spatial mean)

We want to compute the average 2 metre temperature anomaly trend over Central Europe in winter. We already have `t2m_anomaly_central_europe` loaded in memory, we average it by seson and region:

In [ ]:
t2m_anomaly_central_europe_seasonal_and_spatial_mean = t2m_anomaly_central_europe.resample(time="QS-DEC").mean(dim=["time", "latitude", "longitude"])
t2m_anomaly_central_europe_seasonal_and_spatial_mean

Next, we select only the winter values:

In [ ]:
t2m_anomaly_central_europe_winter_spatial_mean = t2m_anomaly_central_europe_seasonal_and_spatial_mean.sel(time=t2m_area_seasonal_spatial_mean["time.month"] == int(12))

We can plot the result:

In [ ]:
plt.plot(t2m_area_winter_mean["time"], t2m_area_winter_mean)
plt.xlabel("Year")
plt.ylabel("Surface Temperature anomaly [C]")
plt.title("Mean Winter surface temperature anomaly in Central Europe", size=10)
plt.show()